In [258]:
# import all the necessary libraries

import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from collections import defaultdict

# %matplotlib inline

In [259]:
# Read the data from a csv to a Pandas dataframe

data_loc = "C:\\Users\\matt\\jupyter-notebooks\\predicting-soccer\\data\\"

raw_data = []

raw_data.append(pd.read_csv(data_loc + '0607-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0708-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0809-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0910-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1011-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1112-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1213-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1314-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1415-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1516-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1617-E0.csv'))


In [260]:
# Parse date string data as time date format

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, "%d/%m/%y").date()
            
for year in raw_data:
    year.Date = year.Date.apply(parse_date)

In [261]:
# Filter the data points related to the gameplay

filtered_data = []
columns_req = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

for year in raw_data:
    filtered_data.append(year[columns_req])

filtered_data[0].head(n=5)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,2006-08-19,Arsenal,Aston Villa,1,1,D
1,2006-08-19,Bolton,Tottenham,2,0,H
2,2006-08-19,Everton,Watford,2,1,H
3,2006-08-19,Newcastle,Wigan,2,1,H
4,2006-08-19,Portsmouth,Blackburn,3,0,H


In [262]:
# Optimise the column naming structure for grouping using pandas groupby method

for i, season in enumerate(filtered_data):
    filtered_data[i].columns = [[    '', 'Home', 'Away', 'Home', 'Away',    ''],
                                ['Date', 'Team', 'Team',  'FTG',  'FTG', 'FTR']]

filtered_data[0].head(n=5)

,,Home,Away,Home,Away,
,Date,Team,Team,FTG,FTG,FTR
0,2006-08-19,Arsenal,Aston Villa,1,1,D
1,2006-08-19,Bolton,Tottenham,2,0,H
2,2006-08-19,Everton,Watford,2,1,H
3,2006-08-19,Newcastle,Wigan,2,1,H
4,2006-08-19,Portsmouth,Blackburn,3,0,H


In [263]:
# Add the cumulative goals scored

def add_cumulative_score_field(frame, cum_field):
    #  Input: 'frame' is a data frame input that has multiple teams to group.
    #         They are grouped by the 'cum_field' string. Teams are either playing home or away.
    # Output: This function has a null return. But it modifies the input frame.
    #         This works as the input frame is a byRef variable asssignment.

    frame_stacked = frame.stack(0)
    total_pts = frame_stacked.groupby('Team')[cum_field].cumsum()

    # Insert the new cumulative sum values in to the frame with the correctly named columns.
    col_heading = "t_scored_{}".format(cum_field)
    frame[('Home', col_heading)] = total_pts[:, 'Home']
    frame[('Away', col_heading)] = total_pts[:, 'Away']
    
    
# Add the cumulative Full Time Scored Goals for both teams.
for i, season in enumerate(filtered_data):
    add_cumulative_field(filtered_data[i], 'FTG')

filtered_data[0]


--> 2.0


c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


--> 0.0
--> 1.0
--> 1.0
--> 0.0
--> 1.0
--> 0.0
--> 1.0
--> 2.0
--> 3.0
--> 1.0


,,Home,Away,Home,Away,,Home,Away
,Date,Team,Team,FTG,FTG,FTR,t_scored_FTG,t_scored_FTG
0,2006-08-19,Arsenal,Aston Villa,1,1,D,1.0,1.0
1,2006-08-19,Bolton,Tottenham,2,0,H,2.0,0.0
2,2006-08-19,Everton,Watford,2,1,H,2.0,1.0
3,2006-08-19,Newcastle,Wigan,2,1,H,2.0,1.0
4,2006-08-19,Portsmouth,Blackburn,3,0,H,3.0,0.0
5,2006-08-19,Reading,Middlesbrough,3,2,H,3.0,2.0
6,2006-08-19,Sheffield United,Liverpool,1,1,D,1.0,1.0
7,2006-08-19,West Ham,Charlton,3,1,H,3.0,1.0
8,2006-08-20,Chelsea,Man City,3,0,H,3.0,0.0


In [265]:
# Add cumulative conceded goals
filtered_data[0]



# Add the cumulative Full Time Conceded Goals for both teams.
# for i, season in enumerate(filtered_data):
#     add_cumulative_field(filtered_data[i], 'FTG', True)

,,Home,Away,Home,Away,,Home,Away
,Date,Team,Team,FTG,FTG,FTR,t_scored_FTG,t_scored_FTG
0,2006-08-19,Arsenal,Aston Villa,1,1,D,1.0,1.0
1,2006-08-19,Bolton,Tottenham,2,0,H,2.0,0.0
2,2006-08-19,Everton,Watford,2,1,H,2.0,1.0
3,2006-08-19,Newcastle,Wigan,2,1,H,2.0,1.0
4,2006-08-19,Portsmouth,Blackburn,3,0,H,3.0,0.0
5,2006-08-19,Reading,Middlesbrough,3,2,H,3.0,2.0
6,2006-08-19,Sheffield United,Liverpool,1,1,D,1.0,1.0
7,2006-08-19,West Ham,Charlton,3,1,H,3.0,1.0
8,2006-08-20,Chelsea,Man City,3,0,H,3.0,0.0


In [172]:
# #*** GOALS SCORED ***
# # Gets the goals scored agg arranged by teams and matchweek
# def get_goals_scored(playing_stat):
#     # Create a dictionary with team names as keys
#     teams = {}
#     for i in playing_stat.groupby('HomeTeam').mean().T.columns:
#         teams[i] = []
    
#     # the value corresponding to keys is a list containing the match location.
#     for i in range(len(playing_stat)):
#         HTGS = playing_stat.iloc[i]['FTHG']
#         ATGS = playing_stat.iloc[i]['FTAG']
#         teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
#         teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
#     # Create a dataframe for goals scored where rows are teams and cols are matchweek.
#     GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
#     GoalsScored[0] = 0
#     # Aggregate to get uptil that point
#     for i in range(2,39):
#         GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
#     return GoalsScored


# #*** GOALS CONCEDED ****   
# # Gets the goals conceded agg arranged by teams and matchweek
# def get_goals_conceded(playing_stat):
#     # Create a dictionary with team names as keys
#     teams = {}
#     for i in playing_stat.groupby('HomeTeam').mean().T.columns:
#         teams[i] = []
    
#     # the value corresponding to keys is a list containing the match location.
#     for i in range(len(playing_stat)):
#         ATGC = playing_stat.iloc[i]['FTHG']
#         HTGC = playing_stat.iloc[i]['FTAG']
#         teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
#         teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
#     # Create a dataframe for goals scored where rows are teams and cols are matchweek.
#     GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
#     GoalsConceded[0] = 0
#     # Aggregate to get uptil that point
#     for i in range(2,39):
#         GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
#     return GoalsConceded

# def get_gss(playing_stat):
#     GC = get_goals_conceded(playing_stat)
#     GS = get_goals_scored(playing_stat)
   
#     j = 0
#     HTGS = []
#     ATGS = []
#     HTGC = []
#     ATGC = []

#     for i in range(380):
#         ht = playing_stat.iloc[i].HomeTeam
#         at = playing_stat.iloc[i].AwayTeam
#         HTGS.append(GS.loc[ht][j])
#         ATGS.append(GS.loc[at][j])
#         HTGC.append(GC.loc[ht][j])
#         ATGC.append(GC.loc[at][j])
        
#         if ((i + 1)% 10) == 0:
#             j = j + 1
        
#     playing_stat['HTGS'] = HTGS
#     playing_stat['ATGS'] = ATGS
#     playing_stat['HTGC'] = HTGC
#     playing_stat['ATGC'] = ATGC
    
#     return playing_stat

# # Apply to the filtered dataset
# for year in filtered_data:
#     year = get_gss(year)
    